In [31]:
import pandas as pd
import tqdm
import bioframe as bf
from collections import Counter
from multiprocessing import Pool
import numpy as np

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [32]:
df = pd.read_csv("gencode.v40.basic.annotation.gtf.gz",
                 sep="\t", skiprows=5, header=None, 
                 names=["chrom", "source", "type", "start", "end", "strand", "data"], 
                 usecols=[0,1,2,3,4,6,8],
                 low_memory=False)

In [33]:
df.head()

,chrom,source,type,start,end,strand,data
0,chr1,HAVANA,gene,11869,14409,+,"gene_id ""ENSG00000223972.5""; gene_type ""transc..."
1,chr1,HAVANA,transcript,11869,14409,+,"gene_id ""ENSG00000223972.5""; transcript_id ""EN..."
2,chr1,HAVANA,exon,11869,12227,+,"gene_id ""ENSG00000223972.5""; transcript_id ""EN..."
3,chr1,HAVANA,exon,12613,12721,+,"gene_id ""ENSG00000223972.5""; transcript_id ""EN..."
4,chr1,HAVANA,exon,13221,14409,+,"gene_id ""ENSG00000223972.5""; transcript_id ""EN..."


In [34]:
genes_df = df[df["type"] == "gene"].reset_index(drop=True)
genes_df.head()

,chrom,source,type,start,end,strand,data
0,chr1,HAVANA,gene,11869,14409,+,"gene_id ""ENSG00000223972.5""; gene_type ""transc..."
1,chr1,HAVANA,gene,14404,29570,-,"gene_id ""ENSG00000227232.5""; gene_type ""unproc..."
2,chr1,ENSEMBL,gene,17369,17436,-,"gene_id ""ENSG00000278267.1""; gene_type ""miRNA""..."
3,chr1,HAVANA,gene,29554,31109,+,"gene_id ""ENSG00000243485.5""; gene_type ""lncRNA..."
4,chr1,ENSEMBL,gene,30366,30503,+,"gene_id ""ENSG00000284332.1""; gene_type ""miRNA""..."


In [35]:
genes_df.shape[0]

61544

In [36]:
gene_metadata = {}
for i in tqdm.trange(genes_df.shape[0]):
    row = genes_df.iloc[i]
    row_metadata = row.data
    row_metadata = row_metadata.replace(';', '').replace('"', '').split()
    metadata = {}
    for i in range(len(row_metadata) // 2):
        metadata[row_metadata[2*i]] = row_metadata[2*i + 1]
    gene_id = metadata["gene_id"]
    gene_type = metadata["gene_type"]
    gene_name = metadata["gene_name"]
    gene_metadata[gene_id] = {"gene_name" : gene_name, "gene_type" : gene_type}

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61544/61544 [00:02<00:00, 22883.15it/s]


In [37]:
transcripts_df = df[df["type"] == "transcript"].reset_index(drop=True)
transcripts_df["gene_id"] = transcripts_df.data.map(lambda x: x.split(";")[0].split()[1].replace('"',''))

In [38]:
transcripts_df.shape

(114816, 8)

In [39]:
transcripts_df.head()

,chrom,source,type,start,end,strand,data,gene_id
0,chr1,HAVANA,transcript,11869,14409,+,"gene_id ""ENSG00000223972.5""; transcript_id ""EN...",ENSG00000223972.5
1,chr1,HAVANA,transcript,12010,13670,+,"gene_id ""ENSG00000223972.5""; transcript_id ""EN...",ENSG00000223972.5
2,chr1,HAVANA,transcript,14404,29570,-,"gene_id ""ENSG00000227232.5""; transcript_id ""EN...",ENSG00000227232.5
3,chr1,ENSEMBL,transcript,17369,17436,-,"gene_id ""ENSG00000278267.1""; transcript_id ""EN...",ENSG00000278267.1
4,chr1,HAVANA,transcript,29554,31097,+,"gene_id ""ENSG00000243485.5""; transcript_id ""EN...",ENSG00000243485.5


In [40]:
transcript_metadata = {}
for i in tqdm.trange(transcripts_df.shape[0]):
    row = transcripts_df.iloc[i]
    row_metadata = row.data
    row_metadata = row_metadata.replace(';', '').replace('"', '').split()
    metadata = {}
    for i in range(len(row_metadata) // 2):
        metadata[row_metadata[2*i]] = row_metadata[2*i + 1]
    gene_id = metadata["gene_id"]
    transcript_id = metadata["transcript_id"]
    gene_name = metadata["gene_name"]
    gene_type = metadata["gene_type"]
    transcript_metadata[transcript_id] = {"gene_id" : gene_id, 
                                          "gene_name" : gene_name,
                                          "gene_type" : gene_type}

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 114816/114816 [00:05<00:00, 20407.86it/s]


In [41]:
transcripts_df.head()

,chrom,source,type,start,end,strand,data,gene_id
0,chr1,HAVANA,transcript,11869,14409,+,"gene_id ""ENSG00000223972.5""; transcript_id ""EN...",ENSG00000223972.5
1,chr1,HAVANA,transcript,12010,13670,+,"gene_id ""ENSG00000223972.5""; transcript_id ""EN...",ENSG00000223972.5
2,chr1,HAVANA,transcript,14404,29570,-,"gene_id ""ENSG00000227232.5""; transcript_id ""EN...",ENSG00000227232.5
3,chr1,ENSEMBL,transcript,17369,17436,-,"gene_id ""ENSG00000278267.1""; transcript_id ""EN...",ENSG00000278267.1
4,chr1,HAVANA,transcript,29554,31097,+,"gene_id ""ENSG00000243485.5""; transcript_id ""EN...",ENSG00000243485.5


In [42]:
def get_TSS(start, end, strand):
    if strand == "+":
        return(start, start)
    else:
        return(end, end)
        
transcripts_df[["TSS_start", "TSS_end"]] = transcripts_df.apply(lambda x: get_TSS(x["start"], x["end"], x["strand"]), axis=1, result_type="expand")

In [43]:
transcripts_df.head()

,chrom,source,type,start,end,strand,data,gene_id,TSS_start,TSS_end
0,chr1,HAVANA,transcript,11869,14409,+,"gene_id ""ENSG00000223972.5""; transcript_id ""EN...",ENSG00000223972.5,11869,11869
1,chr1,HAVANA,transcript,12010,13670,+,"gene_id ""ENSG00000223972.5""; transcript_id ""EN...",ENSG00000223972.5,12010,12010
2,chr1,HAVANA,transcript,14404,29570,-,"gene_id ""ENSG00000227232.5""; transcript_id ""EN...",ENSG00000227232.5,29570,29570
3,chr1,ENSEMBL,transcript,17369,17436,-,"gene_id ""ENSG00000278267.1""; transcript_id ""EN...",ENSG00000278267.1,17436,17436
4,chr1,HAVANA,transcript,29554,31097,+,"gene_id ""ENSG00000243485.5""; transcript_id ""EN...",ENSG00000243485.5,29554,29554


In [44]:
transcripts_df = transcripts_df.rename(columns = {"start" : "transcript_start", 
                                 "end" : "transcript_end", 
                                 "TSS_start" : "start",
                                 "TSS_end" : "end"})

In [45]:
transcripts_df.head()

,chrom,source,type,transcript_start,transcript_end,strand,data,gene_id,start,end
0,chr1,HAVANA,transcript,11869,14409,+,"gene_id ""ENSG00000223972.5""; transcript_id ""EN...",ENSG00000223972.5,11869,11869
1,chr1,HAVANA,transcript,12010,13670,+,"gene_id ""ENSG00000223972.5""; transcript_id ""EN...",ENSG00000223972.5,12010,12010
2,chr1,HAVANA,transcript,14404,29570,-,"gene_id ""ENSG00000227232.5""; transcript_id ""EN...",ENSG00000227232.5,29570,29570
3,chr1,ENSEMBL,transcript,17369,17436,-,"gene_id ""ENSG00000278267.1""; transcript_id ""EN...",ENSG00000278267.1,17436,17436
4,chr1,HAVANA,transcript,29554,31097,+,"gene_id ""ENSG00000243485.5""; transcript_id ""EN...",ENSG00000243485.5,29554,29554


In [46]:
transcript_counts = Counter()
for t in transcript_metadata:
    gene_id = transcript_metadata[t]["gene_id"]
    transcript_counts[gene_id] += 1

In [47]:
transcript_counts.most_common()[:10]

[('ENSG00000109339.24', 87),
 ('ENSG00000145362.21', 70),
 ('ENSG00000121940.17', 62),
 ('ENSG00000109654.16', 53),
 ('ENSG00000234741.9', 52),
 ('ENSG00000164733.22', 52),
 ('ENSG00000087460.29', 50),
 ('ENSG00000228956.9', 44),
 ('ENSG00000134769.23', 44),
 ('ENSG00000182923.20', 41)]

In [48]:
n_most = transcript_counts.most_common()[0][1]

In [49]:
print("A gene could have at most {} transcripts".format(n_most))

A gene could have at most 87 transcripts


In [50]:
cCREs = pd.read_csv("/data/projects/encode/Registry/V4/GRCh38/GRCh38-cCREs.bed",
                    sep="\t",
                    header=None,
                    names=["chrom", "start", "end", "rDHS", "cCRE", "cCRE_type"])
cCREs.head()

,chrom,start,end,rDHS,cCRE,cCRE_type
0,chr1,10033,10250,EH38D4327497,EH38E2776516,pELS
1,chr1,10385,10713,EH38D4327498,EH38E2776517,pELS
2,chr1,16097,16381,EH38D6144701,EH38E3951272,CA-CTCF
3,chr1,17343,17642,EH38D6144702,EH38E3951273,CA-TF
4,chr1,29320,29517,EH38D6144703,EH38E3951274,CA


In [51]:
cCREs = cCREs.sort_values(['chrom', 'start'], ascending=[True, True]).reset_index(drop=True)
transcripts_df = transcripts_df.sort_values(['chrom', 'start'], ascending=[True, True]).reset_index(drop=True)

In [52]:
def get_n_closest(chrom):
    test = bf.closest(cCREs[cCREs["chrom"] == chrom], transcripts_df[transcripts_df["chrom"] == chrom], suffixes=["_cCRE","_transcript"], k=int(3*n_most))
    index = test.drop_duplicates(subset= ["cCRE_cCRE", "gene_id_transcript"]).groupby("cCRE_cCRE").distance.nsmallest(3).index
    index = [_[1] for _ in index]
    return(test.iloc[index])

In [53]:
def run_imap_multiprocessing(func, argument_list, num_processes):

    pool = Pool(processes=num_processes)

    result_list_tqdm = []
    for result in tqdm.tqdm(pool.imap(func=func, iterable=argument_list), total=len(argument_list)):
        result_list_tqdm.append(result)

    return result_list_tqdm

chroms = cCREs.chrom.unique().tolist()
df = run_imap_multiprocessing(get_n_closest, chroms, len(chroms))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [03:55<00:00,  9.80s/it]


In [54]:
df = pd.concat(df).sort_values(["chrom_cCRE", "start_cCRE"], ascending=[True, True]).reset_index(drop=True)

In [55]:
df.head()

,chrom_cCRE,start_cCRE,end_cCRE,rDHS_cCRE,cCRE_cCRE,cCRE_type_cCRE,chrom_transcript,source_transcript,type_transcript,transcript_start_transcript,transcript_end_transcript,strand_transcript,data_transcript,gene_id_transcript,start_transcript,end_transcript,distance
0,chr1,10033,10250,EH38D4327497,EH38E2776516,pELS,chr1,HAVANA,transcript,11869,14409,+,"gene_id ""ENSG00000223972.5""; transcript_id ""EN...",ENSG00000223972.5,11869,11869,1619
1,chr1,10033,10250,EH38D4327497,EH38E2776516,pELS,chr1,ENSEMBL,transcript,17369,17436,-,"gene_id ""ENSG00000278267.1""; transcript_id ""EN...",ENSG00000278267.1,17436,17436,7186
2,chr1,10033,10250,EH38D4327497,EH38E2776516,pELS,chr1,HAVANA,transcript,29554,31097,+,"gene_id ""ENSG00000243485.5""; transcript_id ""EN...",ENSG00000243485.5,29554,29554,19304
3,chr1,10385,10713,EH38D4327498,EH38E2776517,pELS,chr1,HAVANA,transcript,11869,14409,+,"gene_id ""ENSG00000223972.5""; transcript_id ""EN...",ENSG00000223972.5,11869,11869,1156
4,chr1,10385,10713,EH38D4327498,EH38E2776517,pELS,chr1,ENSEMBL,transcript,17369,17436,-,"gene_id ""ENSG00000278267.1""; transcript_id ""EN...",ENSG00000278267.1,17436,17436,6723


In [56]:
gene_names = {_ : gene_metadata[_]["gene_name"] for _ in gene_metadata}

In [57]:
df["gene_name"] = df["gene_id_transcript"].map(gene_names)

In [58]:
columns = ["chrom_cCRE", "start_cCRE", "end_cCRE", "cCRE_cCRE", "rDHS_cCRE", "cCRE_type_cCRE", "gene_id_transcript", "gene_name", "start_transcript", "end_transcript", "distance"]
df[columns].head()

,chrom_cCRE,start_cCRE,end_cCRE,cCRE_cCRE,rDHS_cCRE,cCRE_type_cCRE,gene_id_transcript,gene_name,start_transcript,end_transcript,distance
0,chr1,10033,10250,EH38E2776516,EH38D4327497,pELS,ENSG00000223972.5,DDX11L1,11869,11869,1619
1,chr1,10033,10250,EH38E2776516,EH38D4327497,pELS,ENSG00000278267.1,MIR6859-1,17436,17436,7186
2,chr1,10033,10250,EH38E2776516,EH38D4327497,pELS,ENSG00000243485.5,MIR1302-2HG,29554,29554,19304
3,chr1,10385,10713,EH38E2776517,EH38D4327498,pELS,ENSG00000223972.5,DDX11L1,11869,11869,1156
4,chr1,10385,10713,EH38E2776517,EH38D4327498,pELS,ENSG00000278267.1,MIR6859-1,17436,17436,6723


In [59]:
df[columns].to_csv("GRCh38-cCREs-closest_genes.bed", sep="\t", index=False, header=False)